In [1]:
import os
import os

api_key = os.getenv("OPENAI_API_KEY")

In [4]:
! pip install numexpr

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [17]:
import os
from langchain.chains import LLMMathChain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

chain = LLMMathChain.from_llm(llm=llm)

print(chain.run("2+6等于几？"))

Answer: 8


In [19]:
template = "请用中文解释这个词语：{word}"
prompt = PromptTemplate(input_variables=["word"], template=template)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run({"word": "男人"}))

C:\Users\25746\AppData\Local\Temp\ipykernel_11232\3594235110.py:4: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


男人是指生理上具有男性特征的人类个体，通常具有较强的体力和阳刚气质，也具有独立、坚强、责任感等特质。在社会角色中，男人通常被认为是家庭的经济支柱和保护者，承担着养家糊口、保护家庭成员等责任。男人在社会中扮演着重要的角色，是家庭、社会和国家的支柱之一。


In [21]:
from typing import List, Dict, Any, Optional
from langchain.callbacks.manager import (
    CallbackManagerForChainRun
)
from langchain.chains.base import  Chain
from langchain.prompts.base import BasePromptTemplate
from langchain.base_language import  BaseLanguageModel


In [23]:
class wiki_article_chain(Chain):
    """开发一个wiki文章生成器"""
    prompt:BasePromptTemplate
    llm:BaseLanguageModel
    out_key:str="text"

    @property
    def input_keys(self) -> List[str]:
        """将返回Prompt所需的所有键"""
        return self.prompt.input_variables
    
    @property
    def output_keys(self) -> List[str]:
        """将始终返回text键"""
        return [self.out_key]
    
    def _call(
        self,
        inputs:Dict[str,Any],
        run_manager:Optional[CallbackManagerForChainRun]=None,
    ) -> Dict[str,Any]:
        """运行链"""
        prompt_value = self.prompt.format_prompt(**inputs)
        #print("prompt_value:",prompt_value)
        response = self.llm.generate_prompt(
            [prompt_value],callbacks=run_manager.get_child() if run_manager else None
        )
        #print("response:",response)
        if run_manager:
            run_manager.on_text("wiki article is written")
        return {self.out_key:response.generations[0][0].text}
    
    @property
    def _chain_type(self) -> str:
        """链类型"""
        return "wiki_article_chain"
       

In [ ]:
from langchain.chat_models import  ChatOpenAI
from langchain.prompts import  PromptTemplate

chain = wiki_article_chain(
    prompt=PromptTemplate(
        template="写一篇关于{topic}的维基百科形式的文章",
        input_variables=["topic"]
    ),
    llm=ChatOpenAI(
        temperature=0
    ),
)
result = chain.run({"topic":"降本增效"})
print(result)